In [ ]:
import mxnet as mx
import sagemaker
from sagemaker.mxnet import MXNet as MXNetEstimator

In [ ]:
mx.test_utils.get_cifar10() # Downloads Cifar-10 dataset to ./data

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='data/cifar',
                                       key_prefix='data/cifar10')

In [ ]:
estimator = MXNetEstimator(entry_point='multiple_gpus_sagemaker.py', 
                           role=sagemaker.get_execution_role(),
                           train_instance_count=1, 
                           train_instance_type='ml.p2.8xlarge',
                           hyperparameters={'batch_size': 512, 
                                            'epochs': 30})
estimator.fit(inputs)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.m4.xlarge')

In [ ]:
for i, img in enumerate(image_data):
    response = predictor.predict(img)
    print('image {}: class: {}'.format(i, int(response)))

In [ ]:
estimator.delete_endpoint()